In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
#from scipy.linalg import pinv2
from scipy import linalg
from sklearn.preprocessing import RobustScaler

In [2]:
data = pd.read_csv("iris_data.csv")
data= data.drop(['Unnamed: 0', 'Id'], axis=1)
X=data.drop(['Species'], axis=1)
y = data['Species']
X=X.values
y= y.values
y_2d = y[:, None]
y_one_hot = np.hstack([y_2d==i for i in range(3)]).astype(int)

# Normalize input data
X = RobustScaler().fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

In [4]:
# Compute predictions on training and testing sets
def ELM_L2(l,  X_train, X_test, y_train, y_test,alpha):
    
    Betha=[]
    # Define parameters
    n = X_train.shape[1] #input_size
    k = y_train.shape[1] #output_size

    # Randomly initialize input-to-hidden weights, input_weights
    w = np.random.randn(n, l)

    # Compute hidden layer activations
    #H = np.tanh(X_train@w)
    H = np.maximum(np.dot(X_train, w), 0) # ReLU activation function
    
    # Compute output weights using Moore-Penrose pseudoinverse (with L2 regularization) (B=(H'H)M.H'y)
    B = np.dot(linalg.pinv(H.T.dot(H) + alpha * np.eye(l)), H.T.dot(y_train)) #output_weights

    train_preds = np.dot(H, B)
    
    test_H = np.dot(X_test, w)
    test_H = np.maximum(test_H, 0)  # ReLU activation function
    test_preds = np.dot(test_H, B)
    
    Betha=B
    # Convert predictions to class labels
    train_pred_labels = np.argmax(train_preds, axis=1)
    test_pred_labels = test_preds.argmax(axis=1)
    

    # Calculate accuracy
    train_accuracy = accuracy_score(np.argmax(y_train, axis=1), train_pred_labels)
    test_accuracy = accuracy_score(np.argmax(y_test, axis=1), test_pred_labels)
    
    
    return train_accuracy, test_accuracy, Betha, w
    

In [5]:
for alpha in [0.01,0.1, 1,10, 50, 100]:
    for l in [1, 10, 50, 60, 100]:
        train_accuracy, test_accuracy, B , W = ELM_L2(l, X_train, X_test, y_train, y_test,alpha)
        print(f"L2 alpha={alpha}, neurons = {l} & ELM training score: {train_accuracy:.4f}")
        print(f"Testing Accuracy: {test_accuracy:.4f}")
        print(f"Weight: {W}")
        #print(f"Betha: {B}")
        print()

L2 alpha=0.01, neurons = 1 & ELM training score: 0.3333
Testing Accuracy: 0.3333
Weight: [[-0.14705513]
 [ 1.51166218]
 [-1.83950531]
 [-0.55484832]]

L2 alpha=0.01, neurons = 10 & ELM training score: 0.8583
Testing Accuracy: 0.9000
Weight: [[ 0.50384311 -0.56397706  2.55947945  0.53744769 -0.73659477  1.54492223
  -1.21211951 -0.13169801  0.13078336 -0.99366808]
 [-0.2848235  -2.37084992 -0.31274115 -0.63717788  0.97162187 -2.27445489
   0.65522191  0.3524652  -0.04634832  1.08169365]
 [ 0.41971521  1.36322707  0.2362915   0.20276735 -0.53714075 -0.12163529
   0.77986515  1.30372162 -0.16838579 -0.72253678]
 [-1.60877148  1.75722098 -2.10218835 -1.62042578 -0.94048746  0.01147664
   0.55748385 -1.22343204 -0.21026609  2.37253126]]

L2 alpha=0.01, neurons = 50 & ELM training score: 0.9667
Testing Accuracy: 0.9333
Weight: [[ 2.28871394 -2.26210866 -0.64464405  0.31581086 -1.66113536  1.01898883
  -0.7470757   1.25914222  1.30550463 -0.32539154 -0.55021677 -0.52864452
   2.06244186 -1.63

Save data

In [7]:
np.save("W.npy", W)

In [8]:
W2 = np.load("W.npy")

In [14]:
H2 = np.maximum(np.dot(X_train, W2), 0)

In [15]:
B2 = np.dot(linalg.pinv(H2.T.dot(H2) + alpha * np.eye(l)), H2.T.dot(y_train)) 

Check that the solution is the same

In [19]:
np.allclose(B2, B)

True